In [34]:
import pandas as pd
import openpyxl
from datetime import datetime, timedelta

In [39]:
# LOAD THE EXCEL FILE
df = pd.read_excel("2024-07-15_CE_Schedule.xlsx", sheet_name="CE scheduling")
df.columns

Index(['Start (Hrs) | Event Name | Grps', 'Session Type', '# CEs Needed',
       'Start Date & Time', 'Start Date', 'Duration (hours)', 'Day', 'AM/PM',
       'Sobia Ansari', 'Christina Chen', 'John Butos', 'Elaine Chen',
       'Teresa Davis', 'Caspian Folmsbee', 'Katarzyna Gore', 'Stephen Gore',
       'Anne Hartley', 'Scott Heinrich', 'Suchita Kishore', 'Matthew Kuhns',
       'Carl Lambert', 'Jill Marcus', 'Irena Medenica', 'Sameer Panjwani',
       'Melissa Rice', 'Sara Ruddock-Walker', 'Syed Shah', 'Andrew Simms',
       'Tina Sundarum', 'Paola Susan', 'Frank Tenuto', 'Christine Tsai',
       'Brett Williams', 'Elif Yucebay', 'Rahul Patwari', 'Daven Patel',
       'Viju John', 'Total Column', 'Back Up', 'Scheduler', 'Reviewer',
       'Notes', 'AS Changes', 'ii', 'jj', 'kk', 'll', 'mm', 'nn', 'oo', 'pp'],
      dtype='object')

In [40]:
# GET RID OF UNNEEDED COLUMNS
df.drop(['ii', 'jj','kk', 'll', 'mm', 'nn', 'oo', 'pp', '# CEs Needed'], axis=1, inplace=True)
df.head()

,Start (Hrs) | Event Name | Grps,Session Type,Start Date & Time,Start Date,Duration (hours),Day,AM/PM,Sobia Ansari,Christina Chen,John Butos,...,Elif Yucebay,Rahul Patwari,Daven Patel,Viju John,Total Column,Back Up,Scheduler,Reviewer,Notes,AS Changes
0,08-26-24 1:00pm (2) | M1 PROF Professional Ide...,ROLE,8/26/2024 1:00pm,2024-08-26,2.0,Mon,PM,Yes,Yes,Unavailable,...,Unavailable,Scheduled,NaN,NaN,7.0,K. Gore,AS,NaN,NaN,NaN
1,08-26-24 3:00pm (2) | M1 ADVO/COLL/PROF Small ...,ROLE,8/26/2024 3:00pm,2024-08-26,2.0,Mon,PM,Yes,Unavailable,Unavailable,...,Unavailable,Scheduled,NaN,NaN,7.0,K. Gore,AS,NaN,NaN,NaN
2,08-26-24 9:00am (2) | M1 ADVO EQUI Health Ineq...,ROLE,8/26/2024 9:00am,2024-08-26,2.0,Mon,AM,Scheduled,Scheduled,Unavailable,...,Unavailable,Yes,NaN,NaN,7.0,T. Sundarum,AS,NaN,NaN,NaN
3,08-27-24 10:00am (2) | M1 ADVO EQUI Misuse of ...,ROLE,8/27/2024 10:00am,2024-08-27,2.0,Tue,AM,Scheduled,Unavailable,Top Choice,...,Unavailable,Yes,NaN,NaN,7.0,C. Lambert,AS,This session has 8 CE's Showing as scheduld.,Removed Brett Williams,NaN
4,08-27-24 1:00pm (3) | M1 BSci Intro to Case-ba...,Bsci,8/27/2024 1:00pm,2024-08-27,3.0,Tue,PM,Yes,Unavailable,Scheduled,...,Unavailable,Unavailable,NaN,NaN,7.0,M. Kuhns,AS,NaN,NaN,NaN


In [41]:
# GET THE START AND END datetime FOR THE SESSIONS

# convert 'Start Date & Time' string to a datetime
df['Start Date & Time'] = pd.to_datetime(df['Start Date & Time'])

# convert the duration into a timedelta object
df['delta'] = pd.to_timedelta(df['Duration (hours)'], unit='h')

# create an end datetime column and move it up
df['end datetime'] = df['Start Date & Time'] + df['delta']
df.insert(3, 'end datetime', df.pop('end datetime'))

# rename the start datetime column
df.rename(columns={'Start Date & Time': 'start datetime'}, inplace=True)

df.head()

/var/folders/qs/mmc5bbzd32n4826rw9lqw39m0000gn/T/ipykernel_7267/3194982127.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Start Date & Time'] = pd.to_datetime(df['Start Date & Time'])


,Start (Hrs) | Event Name | Grps,Session Type,start datetime,end datetime,Start Date,Duration (hours),Day,AM/PM,Sobia Ansari,Christina Chen,...,Rahul Patwari,Daven Patel,Viju John,Total Column,Back Up,Scheduler,Reviewer,Notes,AS Changes,delta
0,08-26-24 1:00pm (2) | M1 PROF Professional Ide...,ROLE,2024-08-26 13:00:00,2024-08-26 15:00:00,2024-08-26,2.0,Mon,PM,Yes,Yes,...,Scheduled,NaN,NaN,7.0,K. Gore,AS,NaN,NaN,NaN,0 days 02:00:00
1,08-26-24 3:00pm (2) | M1 ADVO/COLL/PROF Small ...,ROLE,2024-08-26 15:00:00,2024-08-26 17:00:00,2024-08-26,2.0,Mon,PM,Yes,Unavailable,...,Scheduled,NaN,NaN,7.0,K. Gore,AS,NaN,NaN,NaN,0 days 02:00:00
2,08-26-24 9:00am (2) | M1 ADVO EQUI Health Ineq...,ROLE,2024-08-26 09:00:00,2024-08-26 11:00:00,2024-08-26,2.0,Mon,AM,Scheduled,Scheduled,...,Yes,NaN,NaN,7.0,T. Sundarum,AS,NaN,NaN,NaN,0 days 02:00:00
3,08-27-24 10:00am (2) | M1 ADVO EQUI Misuse of ...,ROLE,2024-08-27 10:00:00,2024-08-27 12:00:00,2024-08-27,2.0,Tue,AM,Scheduled,Unavailable,...,Yes,NaN,NaN,7.0,C. Lambert,AS,This session has 8 CE's Showing as scheduld.,Removed Brett Williams,NaN,0 days 02:00:00
4,08-27-24 1:00pm (3) | M1 BSci Intro to Case-ba...,Bsci,2024-08-27 13:00:00,2024-08-27 16:00:00,2024-08-27,3.0,Tue,PM,Yes,Unavailable,...,Unavailable,NaN,NaN,7.0,M. Kuhns,AS,NaN,NaN,NaN,0 days 03:00:00


In [42]:
# NOW GET THE SESSION NAME

# rename the column to an easier to use name
df.rename(columns={'Start (Hrs) | Event Name | Grps': 'bigname'}, inplace=True)

# split on the vertical bar separating each piece 
df[['start (dur)', 'session','Groups']] = df['bigname'].str.split('|',n=2, expand=True)

# move session up front
df.insert(0, 'session', df.pop('session'))

df.head()

,session,bigname,Session Type,start datetime,end datetime,Start Date,Duration (hours),Day,AM/PM,Sobia Ansari,...,Viju John,Total Column,Back Up,Scheduler,Reviewer,Notes,AS Changes,delta,start (dur),Groups
0,M1 PROF Professional Identity Development and...,08-26-24 1:00pm (2) | M1 PROF Professional Ide...,ROLE,2024-08-26 13:00:00,2024-08-26 15:00:00,2024-08-26,2.0,Mon,PM,Yes,...,NaN,7.0,K. Gore,AS,NaN,NaN,NaN,0 days 02:00:00,08-26-24 1:00pm (2),(1-6)
1,M1 ADVO/COLL/PROF Small Group Dynamics,08-26-24 3:00pm (2) | M1 ADVO/COLL/PROF Small ...,ROLE,2024-08-26 15:00:00,2024-08-26 17:00:00,2024-08-26,2.0,Mon,PM,Yes,...,NaN,7.0,K. Gore,AS,NaN,NaN,NaN,0 days 02:00:00,08-26-24 3:00pm (2),(1-6)
2,M1 ADVO EQUI Health Inequities and Implicatio...,08-26-24 9:00am (2) | M1 ADVO EQUI Health Ineq...,ROLE,2024-08-26 09:00:00,2024-08-26 11:00:00,2024-08-26,2.0,Mon,AM,Scheduled,...,NaN,7.0,T. Sundarum,AS,NaN,NaN,NaN,0 days 02:00:00,08-26-24 9:00am (2),(1-6)
3,M1 ADVO EQUI Misuse of Race in Medicine,08-27-24 10:00am (2) | M1 ADVO EQUI Misuse of ...,ROLE,2024-08-27 10:00:00,2024-08-27 12:00:00,2024-08-27,2.0,Tue,AM,Scheduled,...,NaN,7.0,C. Lambert,AS,This session has 8 CE's Showing as scheduld.,Removed Brett Williams,NaN,0 days 02:00:00,08-27-24 10:00am (2),(1-6)
4,M1 BSci Intro to Case-based Curriculum,08-27-24 1:00pm (3) | M1 BSci Intro to Case-ba...,Bsci,2024-08-27 13:00:00,2024-08-27 16:00:00,2024-08-27,3.0,Tue,PM,Yes,...,NaN,7.0,M. Kuhns,AS,NaN,NaN,NaN,0 days 03:00:00,08-27-24 1:00pm (3),(1-6)


In [43]:
df.columns

Index(['session', 'bigname', 'Session Type', 'start datetime', 'end datetime',
       'Start Date', 'Duration (hours)', 'Day', 'AM/PM', 'Sobia Ansari',
       'Christina Chen', 'John Butos', 'Elaine Chen', 'Teresa Davis',
       'Caspian Folmsbee', 'Katarzyna Gore', 'Stephen Gore', 'Anne Hartley',
       'Scott Heinrich', 'Suchita Kishore', 'Matthew Kuhns', 'Carl Lambert',
       'Jill Marcus', 'Irena Medenica', 'Sameer Panjwani', 'Melissa Rice',
       'Sara Ruddock-Walker', 'Syed Shah', 'Andrew Simms', 'Tina Sundarum',
       'Paola Susan', 'Frank Tenuto', 'Christine Tsai', 'Brett Williams',
       'Elif Yucebay', 'Rahul Patwari', 'Daven Patel', 'Viju John',
       'Total Column', 'Back Up', 'Scheduler', 'Reviewer', 'Notes',
       'AS Changes', 'delta', 'start (dur)', 'Groups'],
      dtype='object')

In [45]:
# DROP THE EXTRA COLUMNS
df.drop(['bigname','Start Date','Duration (hours)','AM/PM','delta', 'start (dur)', 'Total Column'], axis=1, inplace=True)
df.head()

,session,Session Type,start datetime,end datetime,Day,Sobia Ansari,Christina Chen,John Butos,Elaine Chen,Teresa Davis,...,Elif Yucebay,Rahul Patwari,Daven Patel,Viju John,Back Up,Scheduler,Reviewer,Notes,AS Changes,Groups
0,M1 PROF Professional Identity Development and...,ROLE,2024-08-26 13:00:00,2024-08-26 15:00:00,Mon,Yes,Yes,Unavailable,Yes,Scheduled,...,Unavailable,Scheduled,NaN,NaN,K. Gore,AS,NaN,NaN,NaN,(1-6)
1,M1 ADVO/COLL/PROF Small Group Dynamics,ROLE,2024-08-26 15:00:00,2024-08-26 17:00:00,Mon,Yes,Unavailable,Unavailable,Yes,Scheduled,...,Unavailable,Scheduled,NaN,NaN,K. Gore,AS,NaN,NaN,NaN,(1-6)
2,M1 ADVO EQUI Health Inequities and Implicatio...,ROLE,2024-08-26 09:00:00,2024-08-26 11:00:00,Mon,Scheduled,Scheduled,Unavailable,Scheduled,If needed,...,Unavailable,Yes,NaN,NaN,T. Sundarum,AS,NaN,NaN,NaN,(1-6)
3,M1 ADVO EQUI Misuse of Race in Medicine,ROLE,2024-08-27 10:00:00,2024-08-27 12:00:00,Tue,Scheduled,Unavailable,Top Choice,Unavailable,If needed,...,Unavailable,Yes,NaN,NaN,C. Lambert,AS,This session has 8 CE's Showing as scheduld.,Removed Brett Williams,NaN,(1-6)
4,M1 BSci Intro to Case-based Curriculum,Bsci,2024-08-27 13:00:00,2024-08-27 16:00:00,Tue,Yes,Unavailable,Scheduled,If needed,If needed,...,Unavailable,Unavailable,NaN,NaN,M. Kuhns,AS,NaN,NaN,NaN,(1-6)


In [46]:
df.to_csv('2024-07-15 CE Schedule.csv')

In [54]:
# FIND THE ROWS WHERE I'M EITHER SCHEDULED OR BACK UP
rp_scheduled = df[(df["Rahul Patwari"]=='Scheduled') | (df["Back Up"]=='R. Patwari')]

In [56]:
df_rp = rp_scheduled[['session', 'start datetime', 'end datetime', 'Session Type','Rahul Patwari','Back Up']]
df_rp

,session,start datetime,end datetime,Session Type,Rahul Patwari,Back Up
0,M1 PROF Professional Identity Development and...,2024-08-26 13:00:00,2024-08-26 15:00:00,ROLE,Scheduled,K. Gore
1,M1 ADVO/COLL/PROF Small Group Dynamics,2024-08-26 15:00:00,2024-08-26 17:00:00,ROLE,Scheduled,K. Gore
8,M1 PRAC Intro to Clinical Reasoning-Groups,2024-08-29 10:00:00,2024-08-29 12:00:00,PRAC,Scheduled,F. Tenuto
16,M2 PRAC/EDUC Clinical Reasoning Rounds with S...,2024-09-09 10:00:00,2024-09-09 12:00:00,SDL/CRR,Scheduled,S. Ruddock-Walker
22,M1 ADVO Social Inequalities and Infectious Di...,2024-09-16 13:00:00,2024-09-16 15:00:00,ROLE,Scheduled,R. Patwari
24,M2 SCHO EBM: Problem Sets Measurement Part 2,2024-09-18 11:00:00,2024-09-18 12:00:00,ROLE,Scheduled,A. Hartley
25,M2 ADVO Trauma-Informed Care and Sexual Viole...,2024-09-18 13:00:00,2024-09-18 15:00:00,ROLE,Scheduled,M. Rice
29,M2 PROF Confidentiality 2-Mature and Emancipa...,2024-09-20 13:00:00,2024-09-20 15:00:00,ROLE,Scheduled,S. Kishore
30,M2 PRAC BCS-2,2024-09-20 08:30:00,2024-09-20 12:00:00,BCS,Scheduled,S. Heinrich
33,M1 BSci Mononucleosis (Katie Diaz),2024-09-23 13:00:00,2024-09-23 16:30:00,Bsci,Scheduled,S. Ruddock-Walker


In [57]:
df_rp.to_csv("rp_schedule_2024-07-15.csv")

In [58]:
# SHOW SEPTEMBER DATES
# top_date = pd.to_datetime('2024-09-01')
# bottom_date = pd.to_datetime('2024-10-01')
# sept_filter = df_rp_scheduled[(df['start datetime']>= top_date) & (df['end datetime'] < bottom_date)]
# sept_filter

In [113]:
sept_filter.sort_values(by="start datetime")

,title,start datetime,end datetime,Session Type,Rahul Patwari
16,M2 PRAC/EDUC Clinical Reasoning Rounds with S...,2024-09-09 10:00:00,2024-09-09 12:00:00,SDL/CRR,Scheduled
22,M1 ADVO Social Inequalities and Infectious Di...,2024-09-16 13:00:00,2024-09-16 15:00:00,ROLE,Scheduled
24,M2 SCHO EBM: Problem Sets Measurement Part 2,2024-09-18 11:00:00,2024-09-18 12:00:00,ROLE,Scheduled
25,M2 ADVO Trauma-Informed Care and Sexual Viole...,2024-09-18 13:00:00,2024-09-18 15:00:00,ROLE,Scheduled
31,M2 PRAC BCS-2,2024-09-20 08:30:00,2024-09-20 12:00:00,BCS,Scheduled
29,M2 PROF Confidentiality 2-Mature and Emancipa...,2024-09-20 13:00:00,2024-09-20 15:00:00,ROLE,Scheduled
32,M1 BSci Mononucleosis (Katie Diaz),2024-09-23 13:00:00,2024-09-23 16:30:00,Bsci,Scheduled
43,M1 BSci Influenza (Kamra Thompson 1),2024-09-30 13:00:00,2024-09-30 16:30:00,Bsci,Scheduled
